In [2]:
import weaviate
import requests
import ast
import json

In [3]:
client = weaviate.Client("http://localhost:8080")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [4]:
schema = {
    "classes": [
    {
        "class": "Question",
        "description": "A question",
        "vectorizer": "text2vec-ollama",
        "moduleConfig": {
            "text2vec-ollama": {
                "apiEndpoint": "http://ollama:11434",
                "model": "nomic-embed-text"
            },
            "generative-ollama": {
                "apiEndpoint": "http://ollama:11434",
                "model": "phi"
            }
        }
    }
    ]
}

In [ ]:
client.schema.create(schema)

In [ ]:
json_string = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')

In [ ]:
datas = ast.literal_eval(json_string.content.decode())

In [ ]:
for data in datas:
    uuid = client.data_object.create(
        class_name = "Question",
        data_object = data
    )

In [4]:
response = (
    client.query
    .get('Question', ['question', 'answer', 'category'])
    .with_near_text({
        "concepts": ['biology']
    })
    .with_limit(2)
    .with_additional(['distance'])
    .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.49546236
          },
          "answer": "Liver",
          "category": "SCIENCE",
          "question": "This organ removes excess glucose from the blood & stores it as glycogen"
        },
        {
          "_additional": {
            "distance": 0.50302213
          },
          "answer": "the atmosphere",
          "category": "SCIENCE",
          "question": "Changes in the tropospheric layer of this are what gives us weather"
        }
      ]
    }
  }
}


In [5]:
generatePrompt = 'Explain {answer} as you might to a five-year-old.'

result = (
  client.query
  .get("Question", ['question', 'answer', 'category'])
  .with_near_text({
    "concepts": ['biology']
  })
  .with_generate(single_prompt=generatePrompt)
  .with_limit(1)
).do()

print(json.dumps(result, indent=2))

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "generate": {
              "error": null,
              "singleResult": " Sure, I'd be happy to explain it in simple terms for a little kid like that. Our liver is like a big factory in our body. It has different parts that do different jobs and work together to help us stay healthy. The liver makes important things like bile (which helps us digest food) and enzymes (which help us break down toxins). If we don't take care of our liver, it can get sick just like any other part of our body. That's why it's important to eat healthy foods and drink plenty of water so our liver can stay strong and keep working properly!\n"
            }
          },
          "answer": "Liver",
          "category": "SCIENCE",
          "question": "This organ removes excess glucose from the blood & stores it as glycogen"
        }
      ]
    }
  }
}


In [6]:
generateTask = "Please answer the question: What is ref2vec? based on the following search results. IMPORTANT! Please make sure your answer is supported by the information in the search results and provide references to the provided content when possible."

result = (
  client.query
  .get("Question", ['question', 'answer', 'category'])
  .with_generate(grouped_task=generateTask)
  .with_near_text({
    "concepts": ["What is ref2vec?"]
  })
  .with_limit(5)
).do()

print(json.dumps(result, indent=2))

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "generate": {
              "error": null,
              "groupedResult": " Ref2vec (or Representation Learning with 2D Vector) is a machine learning algorithm that generates a low-dimensional vector space representation of a set of documents. This representation captures high-level semantic information from the document and can be used for various applications such as text classification, document clustering, and information retrieval.\n\nThe idea behind Ref2vec is to create a vector that represents each word in a document by taking the average of its representations in the context of all other words in the same document. This way, the representation of a document becomes more informative because it captures not only the individual words but also their relationships and co-occurrence patterns.\n\nFor example, consider the following sentence: \"The elephant is an African animal.\" In the vec